In [13]:
from functools import partial
from typing import Callable, List
from modules import dataloaders, schedulers, model, train
from torch import nn, optim
from torchvision import transforms
import torch
import pandas as pd

In [2]:
print(torch.cuda.is_available(), torch.backends.cudnn.enabled)
cuda_flag = torch.cuda.is_available()
device = torch.device('cuda') if cuda_flag else torch.device('cpu')

True True


In [4]:
dl, _ = dataloaders.get_cifar10_data_loaders(data_dir='./data/cifar10', batch_size=100)
rgb_ave = dataloaders.channel_avg(dl)
tfms = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=rgb_ave, std=[1,1,1])])
train_dl, test_dl = dataloaders.get_cifar10_data_loaders(data_dir='./data/cifar10',
                                             batch_size=100,
                                             num_workers=8,
                                             pin_memory=cuda_flag,
                                             train_transform=tfms,
                                             test_transform=tfms)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [5]:
def run_experiment(train_dl:torch.utils.data.DataLoader,
                   test_dl:torch.utils.data.DataLoader,
                   net:nn.Module,
                   optim_groups:List[torch.nn.parameter.Parameter],
                   scheduler_init:Callable[[optim.Optimizer], optim.lr_scheduler._LRScheduler],
                   device:torch.device,
                   iterations:int,
                   result_path:str,
                   criterion:nn.Module=nn.CrossEntropyLoss(),
                   validate_it=1000):
    'TODO: docstring'
    optimizer = optim.SGD(optim_groups, lr=0.001, momentum=0.9, weight_decay=0.004)
    scheduler = scheduler_init(optimizer)
    recorder = {
        'iteration' : [],
        'trn_loss' : [],
        'lr' : [],
        'val_loss' : [],
        'val_acc' : []}
    obe = partial(train.on_batch_end,
                  recorder,
                  test_dl,
                  net,
                  criterion,
                  device,
                  validate_it=validate_it)
    train.train_run(net,
                    train_dl,
                    criterion,
                    optimizer,
                    scheduler,
                    iterations,
                    obe,
                    device)
    return pd.DataFrame(recorder).to_csv(result_path, index=False)

def get_param_groups(net):
    'TODO: docstring.. NB - specific to quick_net'
    conv1_w, conv1_b = net.conv1.parameters()
    conv2_w, conv2_b = net.conv2.parameters()
    conv3_w, conv3_b = net.conv3.parameters()
    ip1_w, ip1_b = net.ip1.parameters()
    ip2_w, ip2_b = net.ip2.parameters()
    pg1 = {'params' : [conv1_w, conv2_w, conv3_w, ip1_w, ip2_w]}
    pg2 = {'params' : [conv1_b, conv2_b, conv3_b, ip1_b, ip2_b], 'lr_mult' : 2.0}
    return [pg1, pg2]

In [12]:
# EXPERIMENT 
# - Cifar10Net_quick
# - mult funk
# - triangular2

    
net = model.Cifar10Net_quick().to(device)
sched_init = partial(schedulers.Triangular2Scheduler, 2000, 0.0025, 0.01)
run_experiment(train_dl,
               test_dl,
               net,
               get_param_groups(net),
               sched_init,
               device,
               26000,
               './results/quick/triangular2.csv')

2.3061602115631104 | 2.3035609197616576 | 0.1002
1.8244595527648926 | 1.7145399641990662 | 0.3677
1.460816502571106 | 1.358925679922104 | 0.5137
1.3165570497512817 | 1.151245186328888 | 0.5892
1.048835277557373 | 1.038002216219902 | 0.6334
1.1155356168746948 | 1.0144459891319275 | 0.6383
1.0659699440002441 | 1.0198210513591766 | 0.6296
0.8306587338447571 | 0.9532518965005875 | 0.6613
0.9904104471206665 | 0.8487013232707977 | 0.7032
0.8551798462867737 | 0.8786535942554474 | 0.692
0.9098547101020813 | 0.8700781047344208 | 0.6941
0.7296966314315796 | 0.7883926743268966 | 0.7198
0.9295708537101746 | 0.8039539659023285 | 0.718
0.6379015445709229 | 0.7916470277309418 | 0.7271
0.6858317852020264 | 0.7840019446611405 | 0.7262
0.8246721625328064 | 0.7546701067686081 | 0.7426
0.7119958996772766 | 0.7426772278547287 | 0.74
0.7968248724937439 | 0.7327794748544693 | 0.7441
0.5366023778915405 | 0.8073395544290543 | 0.717
0.6906335353851318 | 0.7362750527262688 | 0.7472
0.6594592928886414 | 0.7183339

In [11]:
# EXPERIMENT 
# - Cifar10Net_quick
# - mult funk
# - triangular
    
def get_param_groups(net):
    conv1_w, conv1_b = net.conv1.parameters()
    conv2_w, conv2_b = net.conv2.parameters()
    conv3_w, conv3_b = net.conv3.parameters()
    ip1_w, ip1_b = net.ip1.parameters()
    ip2_w, ip2_b = net.ip2.parameters()
    pg1 = {'params' : [conv1_w, conv2_w, conv3_w, ip1_w, ip2_w]}
    pg2 = {'params' : [conv1_b, conv2_b, conv3_b, ip1_b, ip2_b], 'lr_mult' : 2.0}
    return [pg1, pg2]

    
net = model.Cifar10Net_quick().to(device)

sched_init = partial(schedulers.TriangularScheduler, 2000, 0.0025, 0.01)
run_experiment(train_dl,
               test_dl,
               net,
               get_param_groups(net),
               sched_init,
               device,
               24000,
               './results/quick/triangular.csv')

2.298597812652588 | 2.305461356639862 | 0.1
1.6910231113433838 | 1.6804235100746154 | 0.3823
1.426662564277649 | 1.3669135284423828 | 0.5072
1.3729400634765625 | 1.1866989374160766 | 0.5791
1.0422890186309814 | 1.0451254957914353 | 0.6328
1.2875198125839233 | 1.0853311622142792 | 0.614
1.192062258720398 | 1.1166151124238968 | 0.6125
0.8457381725311279 | 0.9440486073493958 | 0.673
0.94527268409729 | 0.8668209141492844 | 0.6991
0.7106298208236694 | 0.8903553664684296 | 0.6897
0.9139995574951172 | 0.882884721159935 | 0.6908
0.7946237921714783 | 0.8180100601911545 | 0.7137
0.5955376625061035 | 0.7593294525146485 | 0.7351
0.6693745255470276 | 0.7968618220090866 | 0.7276
0.7431009411811829 | 0.8545409506559372 | 0.6909
0.7557988166809082 | 0.7632616806030273 | 0.7405
0.678524374961853 | 0.7114196228981018 | 0.7544
0.6242936253547668 | 0.7653144365549087 | 0.7367
0.8155703544616699 | 0.8070110887289047 | 0.7148
0.7658866047859192 | 0.7254400306940079 | 0.7515
0.8359445333480835 | 0.6904992225

In [11]:
# EXPERIMENT 
# - Cifar10Net_quick
# - mult funk
# - triangular
    
def get_param_groups(net):
    conv1_w, conv1_b = net.conv1.parameters()
    conv2_w, conv2_b = net.conv2.parameters()
    conv3_w, conv3_b = net.conv3.parameters()
    ip1_w, ip1_b = net.ip1.parameters()
    ip2_w, ip2_b = net.ip2.parameters()
    pg1 = {'params' : [conv1_w, conv2_w, conv3_w, ip1_w, ip2_w]}
    pg2 = {'params' : [conv1_b, conv2_b, conv3_b, ip1_b, ip2_b], 'lr_mult' : 2.0}
    return [pg1, pg2]

    
net = model.Cifar10Net_quick().to(device)

sched_init = partial(schedulers.TriangularScheduler, 2000, 0.001, 0.006)
run_experiment(train_dl,
               test_dl,
               net,
               get_param_groups(net),
               sched_init,
               device,
               24000,
               './results/quick/triangular.csv')

2.3125171661376953 | 2.306650640964508 | 0.0906
1.958602786064148 | 1.9201850700378418 | 0.2986
1.4172664880752563 | 1.4569110119342803 | 0.4638
1.3893256187438965 | 1.2832274568080901 | 0.5349
1.331922173500061 | 1.193884828686714 | 0.5727
1.2873101234436035 | 1.1909752744436264 | 0.5741
1.0984588861465454 | 1.1049497598409652 | 0.6062
1.0378942489624023 | 1.0078728926181793 | 0.6446
0.9335806965827942 | 0.958577373623848 | 0.6601
1.0462062358856201 | 0.9769250255823135 | 0.655
0.9883946180343628 | 0.9811714136600495 | 0.6553
0.7667956352233887 | 0.8880000388622284 | 0.6845
0.8037870526313782 | 0.8334675049781799 | 0.7056
0.841413676738739 | 0.8691825377941131 | 0.6933
0.8773921132087708 | 0.8657804244756698 | 0.6982
0.7901949882507324 | 0.8330092006921768 | 0.7132
0.7347761392593384 | 0.7628106325864792 | 0.7315
0.8035738468170166 | 0.8518726497888565 | 0.6991
0.7112258076667786 | 0.8274413549900055 | 0.7107
0.6558138132095337 | 0.7726933163404465 | 0.7282
0.7647506594657898 | 0.7308

In [8]:
# EXPERIMENT 
# - Cifar10Net_quick
# - mult funk
# - exp

net = model.Cifar10Net_quick().to(device)
crit = nn.CrossEntropyLoss()
sched_init = partial(schedulers.ExpScheduler, 0.99994, 0.001)

run_experiment(train_dl,
               test_dl,
               net,
               get_param_groups(net),
               sched_init,
               device,
               70000,
               './results/quick/exp.csv')

2.3116936683654785 | 2.304525589942932 | 0.1
2.2790417671203613 | 2.2852141642570496 | 0.1675
1.9101128578186035 | 1.9537005758285522 | 0.2815
1.7431111335754395 | 1.7423493182659149 | 0.3477
1.6380490064620972 | 1.656422470808029 | 0.3786
1.4931074380874634 | 1.5949582076072693 | 0.4128
1.539859652519226 | 1.5306006693840026 | 0.4363
1.4494597911834717 | 1.4924877965450287 | 0.4504
1.58281672000885 | 1.445609645843506 | 0.4707
1.2821382284164429 | 1.4159181141853332 | 0.486
1.25710129737854 | 1.3880424845218657 | 0.4953
1.3438401222229004 | 1.3700767958164215 | 0.5069
1.3636265993118286 | 1.3530028975009918 | 0.5101
1.2750146389007568 | 1.3254806113243103 | 0.5203
1.4875553846359253 | 1.3211042702198028 | 0.5246
1.1813197135925293 | 1.3114898657798768 | 0.5249
1.3227344751358032 | 1.2840170800685882 | 0.54
1.0512477159500122 | 1.2777703189849854 | 0.5403
1.4186445474624634 | 1.2742002022266388 | 0.542
1.3254072666168213 | 1.2557892763614655 | 0.5493
1.2637372016906738 | 1.242764630317

In [6]:
# EXPERIMENT 
# - Cifar10Net_quick
# - mult funk
# - 3 phases
# - fixed
    

net = model.Cifar10Net_quick().to(device)
sched_init = partial(schedulers.FixedScheduler, 0.001)
run_experiment(train_dl,
               test_dl,
               net,
               get_param_groups(net),
               sched_init,
               device,
               60000,
               './results/quick/fixed_1.csv')

sched_init = partial(schedulers.FixedScheduler, 0.0001)
run_experiment(train_dl,
               test_dl,
               net,
               get_param_groups(net),
               sched_init,
               device,
               5000,
               './results/quick/fixed_2.csv')

sched_init = partial(schedulers.FixedScheduler, 0.00001)
run_experiment(train_dl,
               test_dl,
               net,
               get_param_groups(net),
               sched_init,
               device,
               5000,
               './results/quick/fixed_3.csv')

2.2977654933929443 | 2.3063132882118227 | 0.1278
2.2097065448760986 | 2.150905191898346 | 0.2108
2.0062999725341797 | 1.9164109313488007 | 0.2954
1.8871912956237793 | 1.7194612455368041 | 0.366
1.6116914749145508 | 1.583335098028183 | 0.4158
1.4392181634902954 | 1.517972744703293 | 0.4418
1.5057921409606934 | 1.4638865399360657 | 0.4585
1.5817084312438965 | 1.409935268163681 | 0.4819
1.4098000526428223 | 1.376354397535324 | 0.4994
1.3647366762161255 | 1.3372688972949982 | 0.5188
1.2078226804733276 | 1.2971268749237062 | 0.531
1.0433297157287598 | 1.268992087841034 | 0.5433
1.1446075439453125 | 1.2412603086233138 | 0.5587
1.1260707378387451 | 1.1864371514320373 | 0.5805
1.0525401830673218 | 1.1730609720945357 | 0.586
1.2614612579345703 | 1.1515558272600175 | 0.5916
1.0956422090530396 | 1.1268603855371475 | 0.6014
1.151119589805603 | 1.1145139974355698 | 0.604
1.1141318082809448 | 1.080641730427742 | 0.6162
1.0546715259552002 | 1.0606510025262832 | 0.6264
0.9569788575172424 | 1.046824858

0.6710235476493835 | 0.764889766573906 | 0.7375
0.5270580053329468 | 0.7142000064253807 | 0.7561
0.50553297996521 | 0.7140989142656327 | 0.7578
0.6758726239204407 | 0.7186188369989395 | 0.7549
0.7027532458305359 | 0.7125485819578171 | 0.758
0.5635372400283813 | 0.7111390924453735 | 0.7585



0.7453809976577759 | 0.7111451244354248 | 0.7583
0.5968323349952698 | 0.7091064780950547 | 0.7571
0.6399407386779785 | 0.7089621299505233 | 0.7595
0.5697177648544312 | 0.7088916504383087 | 0.7583
0.5112915635108948 | 0.708631392121315 | 0.7583
0.7343107461929321 | 0.7088486209511757 | 0.7591



In [ ]:
# ---- HUMF ----

In [12]:
# EXPERIMENT 
# - Cifar10Net_quick
# - mult funk
# - 3 phases
# - triangular2

def init_weights(net):
    nn.init.normal_(net.conv1.weight.data, std=0.0001, mean=0)
    nn.init.constant_(net.conv1.bias.data, 0)
    nn.init.normal_(net.conv2.weight.data, std=0.01, mean=0)
    nn.init.constant_(net.conv2.bias.data, 0)
    nn.init.normal_(net.conv3.weight.data, std=0.01, mean=0)
    nn.init.constant_(net.conv3.bias.data, 0)
    nn.init.normal_(net.ip1.weight.data, std=0.01, mean=0)
    nn.init.constant_(net.ip1.bias.data, 0)
    nn.init.normal_(net.ip2.weight.data, std=0.01, mean=0)
    nn.init.constant_(net.ip2.bias.data, 0)
    
def get_param_groups(net):
    conv1_w, conv1_b = net.conv1.parameters()
    conv2_w, conv2_b = net.conv2.parameters()
    conv3_w, conv3_b = net.conv3.parameters()
    ip1_w, ip1_b = net.ip1.parameters()
    ip2_w, ip2_b = net.ip2.parameters()
    pg1 = {'params' : [conv1_w, conv2_w, conv3_w, ip1_w, ip2_w]}
    pg2 = {'params' : [conv1_b, conv2_b, conv3_b, ip1_b, ip2_b], 'lr_mult' : 2.0}
    return [pg1, pg2]

    
net = model.Cifar10Net_quick().to(device)
#init_weights(net)

sched_init = partial(schedulers.Triangular2Scheduler, 2000, 0.001, 0.005)
run_experiment(train_dl,
               test_dl,
               net,
               get_param_groups(net),
               sched_init,
               device,
               16000,
               './results/quick/triangular2_1.csv')

sched_init = partial(schedulers.Triangular2Scheduler, 1000, 0.0001, 0.0005)
run_experiment(train_dl,
               test_dl,
               net,
               get_param_groups(net),
               sched_init,
               device,
               6000,
               './results/quick/triangular2_2.csv')

sched_init = partial(schedulers.Triangular2Scheduler, 500, 0.00001, 0.00005)
run_experiment(train_dl,
               test_dl,
               net,
               get_param_groups(net),
               sched_init,
               device,
               3000,
               './results/quick/triangular2_3.csv')

2.307574987411499 | 2.305347640514374 | 0.1
1.7913404703140259 | 1.863457704782486 | 0.3081
1.5311827659606934 | 1.5496428442001342 | 0.428
1.2819905281066895 | 1.3514018380641937 | 0.5067
1.4755597114562988 | 1.2827219533920289 | 0.5376
1.1584140062332153 | 1.2474979984760284 | 0.556
1.1966419219970703 | 1.1830449223518371 | 0.5777
1.1550378799438477 | 1.1190859973430634 | 0.604
0.9551143646240234 | 1.0625218510627747 | 0.6245
1.0678614377975464 | 1.0473969334363937 | 0.6292
1.0332890748977661 | 1.022479500770569 | 0.6392
0.9326040148735046 | 0.9828026992082596 | 0.6569
1.1238511800765991 | 0.945337683558464 | 0.6685
0.9021691083908081 | 0.985873709321022 | 0.6561
0.9842106699943542 | 0.9451193106174469 | 0.672
0.7811145782470703 | 0.9060453110933304 | 0.6812
0.7616575360298157 | 0.9521078234910965 | 0.6688



0.9690132737159729 | 0.9475581270456314 | 0.6706
0.8830466270446777 | 0.8775776654481888 | 0.696
0.6781737804412842 | 0.858720053434372 | 0.704
0.974335789680481 | 0.8710793489217759 | 0.6976
0.7743980288505554 | 0.8546760016679764 | 0.7041
0.9309659600257874 | 0.8520675575733185 | 0.7046
0.8150315880775452 | 0.8502836143970489 | 0.704



0.9172770977020264 | 0.8502853858470917 | 0.704
1.0192338228225708 | 0.8482772099971772 | 0.7075
0.8946125507354736 | 0.8478693985939025 | 0.7078
0.8495210409164429 | 0.8475133973360062 | 0.7072



In [8]:
# EXPERIMENT 
# - Cifar10Net_full
# - mult funk
# - triangular2

def init_weights(net):
    nn.init.normal_(net.conv1.weight.data, std=0.0001, mean=0)
    nn.init.constant_(net.conv1.bias.data, 0)
    nn.init.normal_(net.conv2.weight.data, std=0.01, mean=0)
    nn.init.constant_(net.conv2.bias.data, 0)
    nn.init.normal_(net.conv3.weight.data, std=0.01, mean=0)
    nn.init.constant_(net.conv3.bias.data, 0)
    nn.init.normal_(net.ip1.weight.data, std=0.01, mean=0)
    nn.init.constant_(net.ip1.bias.data, 0)
    
def get_param_groups(net):
    conv1_w, conv1_b = net.conv1.parameters()
    conv2_w, conv2_b = net.conv2.parameters()
    conv3_w, conv3_b = net.conv3.parameters()
    ip1_w, ip1_b = net.ip1.parameters()
    pg1 = {'params' : [conv1_w, conv2_w, conv3_w, conv3_b]}
    pg2 = {'params' : [conv1_b, conv2_b], 'lr_mult' : 2.0}
    pg3 = {'params' : [ip1_w], 'weight_decay' : 1.0}
    pg4 = {'params' : [ip1_b], 'lr_mult' : 2.0, 'weight_decay' : 0.0}
    return [pg1, pg2, pg3, pg4]

    
net = model.Cifar10Net_full().to(device)
#init_weights(net)

sched_init = partial(schedulers.Triangular2Scheduler, 2000, 0.001, 0.005)
run_experiment(train_dl,
               test_dl,
               net,
               get_param_groups(net),
               sched_init,
               device,
               16000,
               './results/full/triangular2_1.csv')

sched_init = partial(schedulers.Triangular2Scheduler, 1000, 0.0001, 0.0005)
run_experiment(train_dl,
               test_dl,
               net,
               get_param_groups(net),
               sched_init,
               device,
               6000,
               './results/full/triangular2_2.csv')

sched_init = partial(schedulers.Triangular2Scheduler, 500, 0.00001, 0.00005)
run_experiment(train_dl,
               test_dl,
               net,
               get_param_groups(net),
               sched_init,
               device,
               3000,
               './results/full/triangular2_3.csv')

2.3008511066436768 | 2.302820382118225 | 0.0999
2.302380323410034 | 2.3023570823669433 | 0.1
2.1252002716064453 | 2.1563481068611146 | 0.217
1.81368088722229 | 1.78774569272995 | 0.3473
1.583215355873108 | 1.7418446946144104 | 0.3359
1.7593404054641724 | 1.7135016453266143 | 0.35
1.6156866550445557 | 1.745770242214203 | 0.3153
1.6110769510269165 | 1.6580060195922852 | 0.3773
1.6276015043258667 | 1.5731531894207 | 0.4178
1.6986182928085327 | 1.6406525111198424 | 0.3796
1.5010789632797241 | 1.5785117197036742 | 0.4108
1.4135628938674927 | 1.5280807065963744 | 0.4309
1.6390644311904907 | 1.5240912306308747 | 0.4352
1.536144733428955 | 1.522464884519577 | 0.4274
1.4629474878311157 | 1.4954882454872132 | 0.451
1.3368349075317383 | 1.4863332533836364 | 0.4489
1.4603351354599 | 1.4728502893447877 | 0.4518


1.5266995429992676 | 1.4722086930274962 | 0.4525
1.4698028564453125 | 1.4347333741188049 | 0.4747
1.2736927270889282 | 1.4258448338508607 | 0.4831
1.539509892463684 | 1.4298111557960511 | 0.4751
1.4489673376083374 | 1.4181612527370453 | 0.4881
1.4453957080841064 | 1.425524938106537 | 0.4793
1.3834052085876465 | 1.4198204016685485 | 0.4832


1.3339438438415527 | 1.4198014438152313 | 0.4829
1.3926093578338623 | 1.4095194935798645 | 0.491
1.3868012428283691 | 1.409732130765915 | 0.4922
1.3620858192443848 | 1.4091048836708069 | 0.4921


In [8]:
# EXPERIMENTS
# - Cifar10Net_full 
# - w/o init or mult funk

# EXPERIMENT: learning rate range test

net = model.Cifar10Net_full().to(device)
model.init_weights(net)
sched_init = partial(schedulers.TriangularScheduler, 4000, 0.001, 0.04)
run_experiment(train_dl,
               test_dl,
               net,
               sched_init,
               device,
               4000,
               './results/full/lrrt.csv',
               validate_it=25)

# EXPERIMENT: fixed learning rate policy

net = model.Cifar10Net_full().to(device)
model.init_weights(net)
run_experiment(train_dl,
               test_dl,
               net,
               schedulers.FixedScheduler,
               device,
               70000,
               './results/full/fixed.csv')

# EXPERIMENT: triangular2 (CLR) learning rate policy 

net = model.Cifar10Net_full().to(device)
model.init_weights(net)
sched_init = partial(schedulers.Triangular2Scheduler, 2000, 0.001, 0.006)

run_experiment(train_dl,
               test_dl,
               net,
               sched_init,
               device,
               25000,
               './results/full/triangular2.csv')

# EXPERIMENT: decay learning rate policy

net = model.Cifar10Net_full().to(device)
model.init_weights(net)
sched_init = partial(schedulers.DecayScheduler, 4000, 0.001, 0.007)
run_experiment(train_dl,
               test_dl,
               net,
               sched_init,
               device,
               25000,
               './results/full/decay.csv')

# EXPERIMENT: exp learning rate policy

net = model.Cifar10Net_full().to(device)
model.init_weights(net)
crit = nn.CrossEntropyLoss()
sched_init = partial(schedulers.ExpScheduler, 0.99994, 0.001)

run_experiment(train_dl,
               test_dl,
               net,
               sched_init,
               device,
               70000,
               './results/full/exp.csv')

# EXPERIMENT: exp_range learning rate policy

net = model.Cifar10Net_full().to(device)
model.init_weights(net)
sched = partial(schedulers.ExpRangeScheduler,0.99994, 2000,0.001, 0.006)
run_experiment(train_dl,
               test_dl,
               net,
               sched_init,
               device,
               42000,
               './results/full/exp_range.csv')

2.30465030670166 | 2.303711314201355 | 0.1
2.2985925674438477 | 2.3035845804214476 | 0.1
2.301647186279297 | 2.3033492517471315 | 0.1
2.3030312061309814 | 2.303114347457886 | 0.1
2.2966582775115967 | 2.302780387401581 | 0.1
2.3018317222595215 | 2.302462773323059 | 0.1
2.299055337905884 | 2.3020982003211974 | 0.1
2.3008315563201904 | 2.3017457056045534 | 0.1
2.299175977706909 | 2.3013187313079833 | 0.101
2.3081653118133545 | 2.300796992778778 | 0.1727
2.2984344959259033 | 2.299976770877838 | 0.1797
2.2962646484375 | 2.2988478207588194 | 0.1723
2.2952635288238525 | 2.2971551036834716 | 0.1785
2.291926860809326 | 2.2945047998428345 | 0.2088
2.2897047996520996 | 2.289475178718567 | 0.1849
2.2819466590881348 | 2.278887469768524 | 0.1594
2.25984525680542 | 2.251675727367401 | 0.1844
2.159640073776245 | 2.178405840396881 | 0.2028
2.2562553882598877 | 2.1036225390434264 | 0.2165
2.023164749145508 | 2.0780276000499724 | 0.2423
2.0153701305389404 | 2.0400673723220826 | 0.2339
1.911908745765686 |

2.285419464111328 | 2.305841493606567 | 0.0985
2.294179677963257 | 2.2940831208229064 | 0.1754
2.099827527999878 | 2.125884523391724 | 0.2152
1.8946256637573242 | 1.9919047474861145 | 0.2633
1.9036554098129272 | 1.8771346306800842 | 0.3271
1.616281270980835 | 1.7252174282073975 | 0.3646
1.6381661891937256 | 1.6438821613788606 | 0.3926
1.5122538805007935 | 1.5747453129291535 | 0.4185
1.4543827772140503 | 1.5425221991539002 | 0.4313
1.6052641868591309 | 1.5154413068294526 | 0.437
1.32778000831604 | 1.4798981201648713 | 0.4539
1.5600225925445557 | 1.4666446721553803 | 0.4574
1.532966136932373 | 1.448083268404007 | 0.4639
1.3878753185272217 | 1.3930725359916687 | 0.4895
1.626566767692566 | 1.3742545688152312 | 0.501
1.3297207355499268 | 1.3633630168437958 | 0.5046
1.2833503484725952 | 1.3447593033313752 | 0.5114
1.2672240734100342 | 1.3279960131645203 | 0.5154
1.2740700244903564 | 1.302495356798172 | 0.53
1.3121875524520874 | 1.2885657680034637 | 0.5365
1.3540982007980347 | 1.2839729022979

2.3024260997772217 | 2.3041430282592774 | 0.1284
2.2621076107025146 | 2.2626279997825622 | 0.1338
1.9237037897109985 | 1.8631570529937744 | 0.3127
1.7503907680511475 | 1.628403615951538 | 0.3981
1.5804166793823242 | 1.4852090322971343 | 0.4548
1.3846451044082642 | 1.393815199136734 | 0.487
1.3365353345870972 | 1.3312723553180694 | 0.5193
1.3161717653274536 | 1.2746328973770142 | 0.5394
1.1548651456832886 | 1.2128922164440155 | 0.5624
1.2272790670394897 | 1.2083317458629608 | 0.5672
1.1041715145111084 | 1.1816556030511856 | 0.5744
1.0698150396347046 | 1.1642704164981843 | 0.5823
1.3718080520629883 | 1.1328082382678986 | 0.5994
0.835521399974823 | 1.0893065184354782 | 0.6143
1.0521982908248901 | 1.0837858998775483 | 0.6128
1.0769633054733276 | 1.0302971243858337 | 0.6347
1.1066813468933105 | 1.0053655725717545 | 0.645
1.0513033866882324 | 1.0100949209928514 | 0.6461
0.9269934892654419 | 1.0163428193330766 | 0.6403
0.9301745891571045 | 0.9833368349075318 | 0.6547
0.9000012874603271 | 0.98

2.324154853820801 | 2.304804606437683 | 0.1
1.5833719968795776 | 1.6935866260528565 | 0.3796
1.5680716037750244 | 1.4983247888088227 | 0.4406
1.4007883071899414 | 1.3816785979270936 | 0.4974
1.383022427558899 | 1.3502704036235809 | 0.5181
1.3079646825790405 | 1.2483495020866393 | 0.5578
1.3469030857086182 | 1.2519806492328645 | 0.5534
1.1645786762237549 | 1.1781806886196136 | 0.5759
1.085235834121704 | 1.1444289678335189 | 0.5932
1.2435857057571411 | 1.1394565600156783 | 0.5976
1.0229257345199585 | 1.1155430746078492 | 0.6074
0.9128177165985107 | 1.1046592098474504 | 0.6097
1.0637434720993042 | 1.10108733355999 | 0.6146
1.149010419845581 | 1.086415228843689 | 0.6191
0.8425028324127197 | 1.0831293439865113 | 0.6186
1.1168345212936401 | 1.073622773885727 | 0.623
1.0694929361343384 | 1.0496599519252776 | 0.6319
1.0994666814804077 | 1.0500125855207443 | 0.6329
1.1207454204559326 | 1.0605770206451417 | 0.6277
1.2234529256820679 | 1.0270198953151704 | 0.644
1.019779920578003 | 1.023195675611

2.3085615634918213 | 2.3046400499343873 | 0.1099
2.2983334064483643 | 2.297817988395691 | 0.1235
2.2057859897613525 | 2.1860960507392884 | 0.1973
2.0078704357147217 | 2.01472727060318 | 0.259
1.932254433631897 | 1.924444212913513 | 0.2965
1.8901968002319336 | 1.8145037651062013 | 0.3266
1.6787623167037964 | 1.7529138314723969 | 0.3472
1.7181183099746704 | 1.6969060564041138 | 0.3724
1.6420494318008423 | 1.6538894176483154 | 0.3932
1.7470887899398804 | 1.6289796221256256 | 0.3955
1.741073489189148 | 1.5828963756561278 | 0.4144
1.6423370838165283 | 1.553854010105133 | 0.4226
1.3950875997543335 | 1.5278353679180146 | 0.433
1.552514910697937 | 1.5147822749614717 | 0.4412
1.444932222366333 | 1.4840609681606294 | 0.4475
1.3846904039382935 | 1.456326196193695 | 0.4644
1.3513189554214478 | 1.4484639322757722 | 0.4698
1.4460252523422241 | 1.434430445432663 | 0.4744
1.3111462593078613 | 1.4051417899131775 | 0.4874
1.4328669309616089 | 1.4225769281387328 | 0.4792
1.452683925628662 | 1.38088703036

2.2953968048095703 | 2.3054661846160887 | 0.1019
2.2832632064819336 | 2.2833551168441772 | 0.1502
2.046269178390503 | 2.0455189824104307 | 0.2407
1.8792619705200195 | 1.9313729548454284 | 0.293
1.7724348306655884 | 1.7871692872047424 | 0.3346
1.868177890777588 | 1.7194798064231873 | 0.3646
1.6476922035217285 | 1.6765387773513794 | 0.3713
1.650820255279541 | 1.6212867403030395 | 0.3974
1.561830997467041 | 1.5636698627471923 | 0.4155
1.5989432334899902 | 1.5548814117908478 | 0.4221
1.6741456985473633 | 1.5175333511829376 | 0.437
1.5801949501037598 | 1.5009349989891052 | 0.4459
1.6154415607452393 | 1.5062439501285554 | 0.4434
1.784041166305542 | 1.4553391027450562 | 0.4655
1.5272709131240845 | 1.4281107985973358 | 0.4764
1.4314515590667725 | 1.419672600030899 | 0.483
1.3089519739151 | 1.401141849756241 | 0.4863
1.473426342010498 | 1.3851660871505738 | 0.4923
1.396676778793335 | 1.381004204750061 | 0.4987
1.376455545425415 | 1.364416400194168 | 0.5043
1.1345033645629883 | 1.353278287649154

In [ ]:
# Cifar10Net_quick EXPERIMENTS

# EXPERIMENT: learning rate range test

net = model.Cifar10Net_quick().to(device)
model.init_weights(net)
sched_init = partial(schedulers.TriangularScheduler, 4000, 0.001, 0.04)
run_experiment(train_dl,
               test_dl,
               net,
               sched_init,
               device,
               4000,
               './results/quick/lrrt.csv',
               validate_it=25)

# EXPERIMENT: fixed learning rate policy

net = model.Cifar10Net_quick().to(device)
model.init_weights(net)
run_experiment(train_dl,
               test_dl,
               net,
               schedulers.FixedScheduler,
               device,
               70000,
               './results/quick/fixed.csv')

# EXPERIMENT: triangular2 (CLR) learning rate policy 

net = model.Cifar10Net_quick().to(device)
model.init_weights(net)
sched_init = partial(schedulers.Triangular2Scheduler, 2000, 0.001, 0.006)

run_experiment(train_dl,
               test_dl,
               net,
               sched_init,
               device,
               25000,
               './results/quick/triangular2_2.csv')

# EXPERIMENT: decay learning rate policy

net = model.Cifar10Net_quick().to(device)
model.init_weights(net)
sched_init = partial(schedulers.DecayScheduler, 4000, 0.001, 0.007)
run_experiment(train_dl,
               test_dl,
               net,
               sched_init,
               device,
               25000,
               './results/quick/decay_2.csv')

# EXPERIMENT: exp learning rate policy

net = model.Cifar10Net_quick().to(device)
model.init_weights(net)
crit = nn.CrossEntropyLoss()
sched_init = partial(schedulers.ExpScheduler, 0.99994, 0.001)

run_experiment(train_dl,
               test_dl,
               net,
               sched_init,
               device,
               70000,
               './results/quick/exp_3.csv')

# EXPERIMENT: exp_range learning rate policy

net = model.Cifar10Net_quick().to(device)
model.init_weights(net)
sched = partial(schedulers.ExpRangeScheduler,0.99994, 2000,0.001, 0.006)
run_experiment(train_dl,
               test_dl,
               net,
               sched_init,
               device,
               42000,
               './results/quick/exp_range_2.csv')

In [ ]:
# TODO: DELETE BELOW

In [9]:
# EXPERIMENT: learning rate range test

net = model.Cifar10Net_quick().to(device)
model.init_weights(net)
sched_init = partial(schedulers.TriangularScheduler, 4000, 0.001, 0.04)
run_experiment(train_dl,
               test_dl,
               net,
               sched_init,
               device,
               4000,
               './results/lrrt2.csv',
               validate_it=25)

2.304516077041626 | 2.3048609638214113 | 0.1054
2.3069167137145996 | 2.304488797187805 | 0.1007
2.3039209842681885 | 2.3039527559280395 | 0.1028
2.295208692550659 | 2.3033598685264587 | 0.1181
2.302865982055664 | 2.3027776288986206 | 0.1088
2.300081253051758 | 2.3019788885116577 | 0.1273
2.295717716217041 | 2.3011337399482725 | 0.1353
2.298647403717041 | 2.3001773381233215 | 0.1606
2.293503761291504 | 2.298734428882599 | 0.1265
2.2935144901275635 | 2.297237000465393 | 0.1499
2.2919416427612305 | 2.2943431997299193 | 0.1296
2.2887930870056152 | 2.2895711350440977 | 0.1286
2.2857391834259033 | 2.281714277267456 | 0.1801
2.2836966514587402 | 2.265476853847504 | 0.1974
2.2485594749450684 | 2.2282635164260864 | 0.2065
2.159878730773926 | 2.1658349990844727 | 0.2033
2.151170253753662 | 2.108147109746933 | 0.2308
2.1795685291290283 | 2.0764776492118835 | 0.2257
2.062422275543213 | 2.0214650619029997 | 0.2495
2.0164906978607178 | 2.011452604532242 | 0.2498
1.8957984447479248 | 2.00157815337181

In [10]:
# EXPERIMENT: fixed learning rate policy

net = model.Cifar10Net_quick().to(device)
model.init_weights(net)
run_experiment(train_dl,
               test_dl,
               net,
               schedulers.FixedScheduler,
               device,
               70000,
               'fixed2.csv')

2.292389392852783 | 2.305676529407501 | 0.1
2.2962806224823 | 2.301499183177948 | 0.1009
2.2934370040893555 | 2.2942102193832397 | 0.1704
2.1653144359588623 | 2.1828091287612916 | 0.2097
1.927932620048523 | 1.991353269815445 | 0.2674
1.8914573192596436 | 1.854973202943802 | 0.3144
1.6109402179718018 | 1.7514321291446686 | 0.3598
1.5970404148101807 | 1.672984594106674 | 0.3814
1.6003726720809937 | 1.622306238412857 | 0.3915
1.548377275466919 | 1.5781875145435333 | 0.4125
1.6007107496261597 | 1.5399190866947174 | 0.4292
1.4526408910751343 | 1.5022450697422027 | 0.4417
1.4007147550582886 | 1.4866573119163513 | 0.4504
1.3464469909667969 | 1.444975861310959 | 0.4614
1.4099258184432983 | 1.4501166594028474 | 0.4641
1.5757185220718384 | 1.3966692447662354 | 0.4834
1.3880709409713745 | 1.3730267000198364 | 0.4972
1.3430217504501343 | 1.360551611185074 | 0.5023
1.4896854162216187 | 1.3289223742485046 | 0.5183
1.177941918373108 | 1.3132964408397674 | 0.5232
1.2616137266159058 | 1.290154372453689

In [11]:
# EXPERIMENT: triangular2 (CLR) learning rate policy 

net = model.Cifar10Net_quick().to(device)
model.init_weights(net)
sched_init = partial(schedulers.Triangular2Scheduler, 2000, 0.001, 0.006)

run_experiment(train_dl,
               test_dl,
               net,
               sched_init,
               device,
               25000,
               './results/triangular2_2.csv')

2.2968926429748535 | 2.30457909822464 | 0.1
2.2868664264678955 | 2.286785531044006 | 0.197
1.9440335035324097 | 1.8334182965755463 | 0.3164
1.7572338581085205 | 1.6157864546775818 | 0.4007
1.5581908226013184 | 1.5078701889514923 | 0.4485
1.358525037765503 | 1.3846300208568574 | 0.4987
1.4854240417480469 | 1.2941029393672943 | 0.5328
1.1617599725723267 | 1.24062604367733 | 0.5534
1.2970151901245117 | 1.2233579629659652 | 0.5633
1.0547809600830078 | 1.1899817889928819 | 0.5721
1.2854118347167969 | 1.2040015226602554 | 0.5681
1.2484716176986694 | 1.1965202814340592 | 0.5689
1.1591776609420776 | 1.1221379524469375 | 0.6
0.9968650341033936 | 1.0945366656780242 | 0.6129
1.1042494773864746 | 1.060657023191452 | 0.6247
1.0013238191604614 | 1.0282442647218704 | 0.6414
1.151883602142334 | 1.0122625637054443 | 0.6456
1.0135200023651123 | 1.0158803641796113 | 0.6443
1.0705336332321167 | 0.996675221323967 | 0.6509
0.762789785861969 | 1.0033236515522004 | 0.6489
0.9662367105484009 | 0.98576820611953

In [12]:
# EXPERIMENT: decay learning rate policy

net = model.Cifar10Net_quick().to(device)
model.init_weights(net)
sched_init = partial(schedulers.DecayScheduler, 4000, 0.001, 0.007)
run_experiment(train_dl,
               test_dl,
               net,
               sched_init,
               device,
               25000,
               './results/decay_2.csv')

2.3068320751190186 | 2.305737099647522 | 0.1
1.8455226421356201 | 1.741349081993103 | 0.341
1.3168511390686035 | 1.5426330029964448 | 0.4306
1.2807995080947876 | 1.4034880530834197 | 0.4856
1.3582720756530762 | 1.3313095653057099 | 0.5173
1.4101901054382324 | 1.2592725324630738 | 0.5426
1.1797466278076172 | 1.2313115227222442 | 0.556
1.3030451536178589 | 1.1897713398933412 | 0.5725
1.2574032545089722 | 1.1579323160648345 | 0.5857
1.0271835327148438 | 1.1512867194414138 | 0.5916
1.120173692703247 | 1.1208481669425965 | 0.5996
1.1658633947372437 | 1.1313316065073014 | 0.598
1.0303394794464111 | 1.0943329101800918 | 0.6124
1.1335058212280273 | 1.0864201718568802 | 0.6163
1.1425772905349731 | 1.1019526296854019 | 0.6139
1.238189935684204 | 1.105303453207016 | 0.6112
1.2152143716812134 | 1.066099419593811 | 0.6215
0.9749917387962341 | 1.0517398589849472 | 0.6299
1.1790443658828735 | 1.0545223528146743 | 0.6269
1.1043736934661865 | 1.0503594851493836 | 0.6256
1.143110990524292 | 1.0239750176

In [ ]:
# EXPERIMENT: exp learning rate policy

net = model.Cifar10Net_quick().to(device)
model.init_weights(net)
crit = nn.CrossEntropyLoss()
sched_init = partial(schedulers.ExpScheduler, 0.99994, 0.001)

run_experiment(train_dl,
               test_dl,
               net,
               sched_init,
               device,
               70000,
               './results/exp_3.csv')

In [ ]:
# EXPERIMENT: exp_range learning rate policy

net = model.Cifar10Net_quick().to(device)
model.init_weights(net)
sched = partial(schedulers.ExpRangeScheduler,0.99994, 2000,0.001, 0.006)
run_experiment(train_dl,
               test_dl,
               net,
               sched_init,
               device,
               42000,
               './results/exp_range_2.csv')

2.3033275604248047 | 2.3048804426193237 | 0.125
1.7304213047027588 | 1.7981905615329743 | 0.3349
1.6655535697937012 | 1.5733483064174651 | 0.4171
1.2522352933883667 | 1.4444288444519042 | 0.4615
1.2984336614608765 | 1.373805477619171 | 0.5014
1.1318652629852295 | 1.3159936928749085 | 0.5188
1.2138344049453735 | 1.2724424839019775 | 0.5443
1.2376002073287964 | 1.2243223750591279 | 0.5564
1.1638312339782715 | 1.2007113236188889 | 0.5742
1.2266716957092285 | 1.1739650183916093 | 0.5832
1.2556490898132324 | 1.1645180624723435 | 0.5815
1.066576600074768 | 1.1505553632974626 | 0.594
1.0201787948608398 | 1.1410143041610719 | 0.5931
0.980394721031189 | 1.1188978105783463 | 0.6053
1.1203137636184692 | 1.1156153786182403 | 0.6074
1.0896766185760498 | 1.10135098695755 | 0.613
1.017928957939148 | 1.0824932366609574 | 0.6237
1.1794638633728027 | 1.0765542471408844 | 0.6213
0.9724977612495422 | 1.0764544117450714 | 0.6193
1.0801877975463867 | 1.0585443252325057 | 0.6263
1.2273309230804443 | 1.045135

In [21]:
# TODO: DELETE

class DemoNet(nn.Module):
    'Port of caffe architecture: https://github.com/BVLC/caffe/blob/master/examples/cifar10/cifar10_quick_train_test.prototxt'

    def __init__(self, num_classes=10):
        'TODO: docstring'
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, padding=2)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, padding=2)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.AvgPool2d(kernel_size=3, stride=2)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=2)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.AvgPool2d(kernel_size=3, stride=2)
        self.ip1 = nn.Linear(64 * 3 * 3, 64)
        self.ip2 = nn.Linear(64, 10)

    def forward(self, x):
        'TODO: docstring'
        out = self.conv1(x)
        out = self.pool1(out)
        out = self.relu1(out)
        out = self.conv2(out)
        out = self.relu2(out)
        out = self.pool2(out)
        out = self.conv3(out)
        out = self.relu3(out)
        out = self.pool3(out)
        out = self.ip1(out.view(out.shape[0], out.shape[1] * out.shape[2] * out.shape[3]))
        out = self.ip2(out)
        return out


inp = next(iter(train_dl))[0]
net = DemoNet()
net(inp).shape

torch.Size([100, 10])

In [ ]:
 #  ------- LEGACY -------

In [ ]:

def record_run(train_dl:torch.utils.data.DataLoader,
               test_dl:torch.utils.data.DataLoader,
               net:nn.Module,
               crit:nn.Module,
               optimizer:optim.Optimizer,
               scheduler:optim.lr_scheduler._LRScheduler,
               device:torch.device,
               iterations:int,
               result_path:str,
               validate_it=500):
    'TODO:docstring'
    recorder = {
        'iteration' : [],
        'trn_loss' : [],
        'lr' : [],
        'val_loss' : [],
        'val_acc' : []}
    obe = partial(train.on_batch_end, recorder, test_dl, net, crit, device, validate_it=validate_it)
    train.train_run(net, train_dl, crit, opt, scheduler, iterations, obe, device)
    return pd.DataFrame(recorder).to_csv(result_path, index=False)

In [6]:
# learning rate range test
tfms = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=rgb_ave, std=[1,1,1])])
train_dl, test_dl = dataloaders.get_cifar10_data_loaders(data_dir='./data/cifar10',
                                             batch_size=100,
                                             num_workers=8,
                                             pin_memory=cuda_flag,
                                             train_transform=tfms,
                                             test_transform=tfms)

net = model.Cifar10Net_quick().to(device)
model.init_weights(net)
crit = nn.CrossEntropyLoss()
opt = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=0.004)
sched = schedulers.TriangularScheduler(4000, 0.001, 0.04, opt)
record_run(train_dl,
           test_dl,
           net,
           crit,
           opt,
           sched,
           device,
           4000,
           './results/lrrt2.csv',
           25)

Files already downloaded and verified
Files already downloaded and verified


2.297976016998291 | 2.3042796659469604 | 0.1
2.309221029281616 | 2.303995385169983 | 0.1
2.30049204826355 | 2.3035417294502256 | 0.1
2.299346685409546 | 2.3030977058410644 | 0.1
2.3032021522521973 | 2.302693500518799 | 0.1
2.3029372692108154 | 2.3023087000846862 | 0.1
2.3073477745056152 | 2.301745216846466 | 0.1
2.3032102584838867 | 2.3009733629226683 | 0.1
2.293759346008301 | 2.3001813006401064 | 0.1005
2.298952341079712 | 2.299080331325531 | 0.1007
2.3037867546081543 | 2.297426505088806 | 0.1068
2.2963027954101562 | 2.294880123138428 | 0.1372
2.2922909259796143 | 2.290413691997528 | 0.1531
2.2857234477996826 | 2.2822949862480164 | 0.1855
2.2633185386657715 | 2.2613956046104433 | 0.1829
2.2346177101135254 | 2.2079072213172912 | 0.1883
2.1203038692474365 | 2.129856433868408 | 0.2057
2.1327919960021973 | 2.0665711760520935 | 0.2266
2.0027883052825928 | 2.0484903216362 | 0.2313
2.0461008548736572 | 2.0263922429084777 | 0.24
1.928764820098877 | 2.0085027587413786 | 0.2555
1.97331178188323

In [19]:
# fixed
tfms = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=rgb_ave, std=[1,1,1])])
train_dl, test_dl = dataloaders.get_cifar10_data_loaders(data_dir='./data/cifar10',
                                             batch_size=100,
                                             num_workers=8,
                                             pin_memory=cuda_flag,
                                             train_transform=tfms,
                                             test_transform=tfms)

net = model.Cifar10Net().to(device)
model.init_weights(net)
crit = nn.CrossEntropyLoss()
opt = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=0.004)
sched = schedulers.FixedScheduler(optimizer=opt)
record_run(train_dl,
           test_dl,
           net,
           crit,
           opt,
           sched,
           device,
           70000,
           './results/fixed.csv')

Files already downloaded and verified
Files already downloaded and verified


2.2959251403808594 | 2.3048656678199766 | 0.1
2.2366421222686768 | 2.252965657711029 | 0.2115
1.9379587173461914 | 1.9531116092205048 | 0.2997
1.8152180910110474 | 1.7316903233528138 | 0.3808
1.6335580348968506 | 1.598079216480255 | 0.4195
1.3986411094665527 | 1.522326169013977 | 0.4467
1.389978289604187 | 1.475188820362091 | 0.4663
1.5397993326187134 | 1.4426518404483795 | 0.4792
1.3852248191833496 | 1.3794143879413605 | 0.5059
1.40769624710083 | 1.3441275775432586 | 0.521
1.2174060344696045 | 1.3135958635807037 | 0.5323
1.3884704113006592 | 1.2910792154073716 | 0.5377
1.1617053747177124 | 1.2626908934116363 | 0.5492
1.2419323921203613 | 1.2411016094684602 | 0.5572
0.9752600789070129 | 1.2126768308877944 | 0.5731
1.1910371780395508 | 1.2149149960279464 | 0.569
1.2233253717422485 | 1.1825607550144195 | 0.5805
1.1781970262527466 | 1.2094527328014373 | 0.5747
1.2020012140274048 | 1.1470334786176681 | 0.5952
1.2650545835494995 | 1.1130068135261535 | 0.6075
1.0533565282821655 | 1.144056512

In [8]:
# triangular 2
tfms = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=rgb_ave, std=[1,1,1])])
train_dl, test_dl = dataloaders.get_cifar10_data_loaders(data_dir='./data/cifar10',
                                             batch_size=100,
                                             num_workers=8,
                                             pin_memory=cuda_flag,
                                             train_transform=tfms,
                                             test_transform=tfms)

net = model.Cifar10Net_quick().to(device)
model.init_weights(net)
crit = nn.CrossEntropyLoss()
opt = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9, weight_decay=0.004)
sched = schedulers.Triangular2Scheduler(step_size=2000, min_lr=0.001, max_lr=0.006, optimizer=opt)
record_run(train_dl,
           test_dl,
           net,
           crit,
           opt,
           sched,
           device,
           25000,
           './results/triangular2_2.csv')

Files already downloaded and verified
Files already downloaded and verified


2.3118653297424316 | 2.305169324874878 | 0.1
2.2890708446502686 | 2.289864807128906 | 0.1631
1.8108723163604736 | 1.889981905221939 | 0.3069
1.554026484489441 | 1.6907937335968017 | 0.3728
1.5032235383987427 | 1.481263747215271 | 0.4569
1.3109405040740967 | 1.4180316984653474 | 0.4863
1.328588604927063 | 1.2947986006736756 | 0.5335
1.1822865009307861 | 1.2496675336360932 | 0.5459
1.0985366106033325 | 1.1947410076856613 | 0.5725
1.110060691833496 | 1.2258172476291656 | 0.5624
1.214227318763733 | 1.1856699573993683 | 0.5734
1.2990254163742065 | 1.167282493710518 | 0.5839
1.0115678310394287 | 1.157571920156479 | 0.5914
1.1854610443115234 | 1.0996272665262223 | 0.6075
1.1256828308105469 | 1.0671267771720887 | 0.6234
1.1380622386932373 | 1.0444374054670333 | 0.6326
0.8972554802894592 | 1.009858409166336 | 0.6486
1.071244239807129 | 1.0174080002307893 | 0.6416
1.0800983905792236 | 1.0005807238817215 | 0.6503
1.025336742401123 | 1.0176855820417403 | 0.6436
1.0522180795669556 | 0.9715644073486

In [16]:
# decay
tfms = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=rgb_ave, std=[1,1,1])])
train_dl, test_dl = dataloaders.get_cifar10_data_loaders(data_dir='./data/cifar10',
                                             batch_size=100,
                                             num_workers=8,
                                             pin_memory=cuda_flag,
                                             train_transform=tfms,
                                             test_transform=tfms)

net = model.Cifar10Net().to(device)
model.init_weights(net)
crit = nn.CrossEntropyLoss()
opt = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9, weight_decay=0.004)
sched = schedulers.DecayScheduler(step_size=4000, min_lr=0.001, max_lr=0.007, optimizer=opt)
record_run(train_dl,
           test_dl,
           net,
           crit,
           opt,
           sched,
           device,
           25000,
           './results/decay.csv')

Files already downloaded and verified
Files already downloaded and verified


2.3158750534057617 | 2.3052439451217652 | 0.0939
1.55889892578125 | 1.5285473811626433 | 0.4411
1.4037697315216064 | 1.3739276313781739 | 0.5096
1.0877312421798706 | 1.2430369985103606 | 0.5566
1.0989784002304077 | 1.1535150969028474 | 0.6035
1.1141140460968018 | 1.0960009902715684 | 0.6123
1.0204178094863892 | 1.0451411986351014 | 0.6345
1.0428575277328491 | 1.0014646846055983 | 0.6501
1.1275478601455688 | 0.9752665412425995 | 0.6587
0.8905993700027466 | 0.9625169438123703 | 0.6675
0.9804762005805969 | 0.958123340010643 | 0.664
0.8910219073295593 | 0.9456889796257019 | 0.671
1.0942773818969727 | 0.9407467526197434 | 0.6734
0.8029584288597107 | 0.9291936069726944 | 0.6726
0.8100183606147766 | 0.9213050878047944 | 0.6782
0.9432716965675354 | 0.9157968461513519 | 0.6824
0.9402356147766113 | 0.9094270581007003 | 0.6816
0.7231302857398987 | 0.9107110375165939 | 0.6791
0.7717517018318176 | 0.9013521921634674 | 0.6847
0.9524957537651062 | 0.8880109769105912 | 0.6909
0.8856656551361084 | 0.89

In [13]:
# exp
tfms = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=rgb_ave, std=[1,1,1])])
train_dl, test_dl = dataloaders.get_cifar10_data_loaders(data_dir='./data/cifar10',
                                             batch_size=100,
                                             num_workers=8,
                                             pin_memory=cuda_flag,
                                             train_transform=tfms,
                                             test_transform=tfms)

net = model.Cifar10Net().to(device)
model.init_weights(net)
crit = nn.CrossEntropyLoss()
opt = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9, weight_decay=0.004)
sched = schedulers.ExpScheduler(gamma= 0.99994, initial_lr=0.001, optimizer=opt)
record_run(train_dl,
           test_dl,
           net,
           crit,
           opt,
           sched,
           device,
           70000,
           './results/exp_2.csv')

Files already downloaded and verified
Files already downloaded and verified


2.3062984943389893 | 2.304504623413086 | 0.1
2.039564609527588 | 2.1313498187065125 | 0.2341
1.9713293313980103 | 1.9188920211791993 | 0.315
1.667871117591858 | 1.722483547925949 | 0.3802
1.608794093132019 | 1.6095246386528015 | 0.4151
1.4804493188858032 | 1.5190627157688141 | 0.4431
1.4638692140579224 | 1.4656637454032897 | 0.466
1.3903836011886597 | 1.4305872213840485 | 0.4819
1.1998200416564941 | 1.4089925062656403 | 0.4867
1.3671257495880127 | 1.372826428413391 | 0.5058
1.4028111696243286 | 1.346828166246414 | 0.5166
1.2423450946807861 | 1.3199831914901734 | 0.5302
1.1036854982376099 | 1.320092191696167 | 0.5275
1.2234302759170532 | 1.2978788781166077 | 0.541
1.2314274311065674 | 1.2897173726558686 | 0.5384
1.1907132863998413 | 1.247455985546112 | 0.56
1.2181692123413086 | 1.2384896421432494 | 0.5607
1.1905916929244995 | 1.2258339285850526 | 0.5674
1.1279654502868652 | 1.2145337605476378 | 0.5695
1.3476121425628662 | 1.2191427284479142 | 0.5657
1.1123565435409546 | 1.19009255766868

In [14]:
# exp_range
tfms = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=rgb_ave, std=[1,1,1])])
train_dl, test_dl = dataloaders.get_cifar10_data_loaders(data_dir='./data/cifar10',
                                             batch_size=100,
                                             num_workers=8,
                                             pin_memory=cuda_flag,
                                             train_transform=tfms,
                                             test_transform=tfms)

net = model.Cifar10Net().to(device)
model.init_weights(net)
crit = nn.CrossEntropyLoss()
opt = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=0.004)
sched = schedulers.ExpRangeScheduler(gamma=0.99994, step_size=2000, min_lr=0.001, max_lr=0.006, optimizer=opt)
record_run(train_dl,
           test_dl,
           net,
           crit,
           opt,
           sched,
           device,
           42000,
           './results/exp_range_2.csv')

Files already downloaded and verified
Files already downloaded and verified


2.3022027015686035 | 2.3050201654434206 | 0.1
2.094893455505371 | 2.0143145072460173 | 0.2726
1.4727064371109009 | 1.60831512093544 | 0.4069
1.4026302099227905 | 1.4376909470558166 | 0.4797
1.4212661981582642 | 1.321196415424347 | 0.5283
1.3423069715499878 | 1.223569244146347 | 0.5675
1.3500639200210571 | 1.1336965662240983 | 0.6035
0.9937301874160767 | 1.0745982599258423 | 0.626
0.9549305438995361 | 1.0473684185743333 | 0.6372
1.0692508220672607 | 1.035099944472313 | 0.6449
1.079912781715393 | 1.0299227404594422 | 0.6426
0.9949414134025574 | 1.040691353082657 | 0.6359
0.8612189292907715 | 0.9839828884601594 | 0.6582
0.9411316514015198 | 0.9726246029138566 | 0.6592
1.0404269695281982 | 0.9285584843158722 | 0.6766
0.9078772068023682 | 0.8814988321065903 | 0.6983
0.6949732899665833 | 0.8608512270450592 | 0.7049
0.7867686748504639 | 0.8890821141004562 | 0.6884
0.7824012041091919 | 0.879164976477623 | 0.7014
0.9796743988990784 | 0.8612118291854859 | 0.7041
0.9537234306335449 | 0.8866164934